# Description

**UPDATE**

This notebook is similar to `30-gls_on_phenotypes-phenomexcan.ipynb` but for eMERGE S-MultiXcan results instead of PhenomeXcan.

Since we don't have partition/clusters of eMERGE results, we selected the same set of LVs from the run on PhenomeXcan (`30-gls_on_phenotypes-phenomexcan.ipynb`); and regarding traits, we take the top 20 traits from each LVs to create a list of trait/LVs pairs to run GLSPhenoplier on.

# Environment variables

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import conf

In [3]:
N_JOBS = conf.GENERAL["N_JOBS"]
display(N_JOBS)

1

In [4]:
%env MKL_NUM_THREADS=$N_JOBS
%env OPEN_BLAS_NUM_THREADS=$N_JOBS
%env NUMEXPR_NUM_THREADS=$N_JOBS
%env OMP_NUM_THREADS=$N_JOBS

env: MKL_NUM_THREADS=1
env: OPEN_BLAS_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: OMP_NUM_THREADS=1


# Modules

In [5]:
from pathlib import Path

import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
from tqdm import tqdm

from gls import GLSPhenoplier

# Settings

In [6]:
OUTPUT_DIR = conf.RESULTS["GLS"]
display(OUTPUT_DIR)

OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier-asthma/base/results/gls')

In [7]:
OUTPUT_FILENAME = OUTPUT_DIR / "gls_phenotypes-emerge-asthma.pkl"
display(OUTPUT_FILENAME)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier-asthma/base/results/gls/gls_phenotypes-emerge-asthma.pkl')

In [8]:
PHENOMEXCAN_INPUT_FILE = conf.RESULTS["GLS"] / "gls_phenotypes-phenomexcan-asthma.pkl"
display(PHENOMEXCAN_INPUT_FILE)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier-asthma/base/results/gls/gls_phenotypes-phenomexcan-asthma.pkl')

# Load data

## eMERGE traits info

In [9]:
# FIXME: in the future, there will be a specific entry in config for the eMERGE directory that should be replaced here
input_filepath = Path(
    conf.DATA_DIR,
    "emerge",
    "eMERGE_III_PMBB_GSA_v2_2020_phecode_AFR_EUR_cc50_counts_w_dictionary.txt",
).resolve()
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/data/emerge/eMERGE_III_PMBB_GSA_v2_2020_phecode_AFR_EUR_cc50_counts_w_dictionary.txt')

In [10]:
emerge_traits_df = pd.read_csv(
    input_filepath,
    sep="\t",
    dtype={"phecode": str},
    usecols=["phecode", "phenotype", "category"],
)

In [11]:
emerge_traits_df = emerge_traits_df.rename(
    columns={
        "phenotype": "phecode_phenotype",
        "category": "phecode_category",
    }
)

In [12]:
emerge_traits_df.shape

(309, 3)

In [13]:
emerge_traits_df.head()

,phecode,phecode_phenotype,phecode_category
0,008,Intestinal infection,infectious diseases
1,008.5,Bacterial enteritis,infectious diseases
2,008.52,Intestinal infection due to C. difficile,infectious diseases
3,038,Septicemia,infectious diseases
4,038.3,Bacteremia,infectious diseases


## eMERGE (S-MultiXcan) projection

In [14]:
input_filepath = Path(
    conf.RESULTS["PROJECTIONS_DIR"],
    "projection-emerge-smultixcan-mashr-zscores.pkl",
).resolve()
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/projections/projection-emerge-smultixcan-mashr-zscores.pkl')

In [15]:
emerge_projection = pd.read_pickle(input_filepath)

In [16]:
emerge_projection.shape

(987, 309)

In [17]:
emerge_projection.head()

,292.3,079,741,418.1,280,747,591,426.21,244,070.3,...,427,585.4,345,366.2,420.2,195,250.2,250.1,371,562
LV1,0.055748,-0.019274,0.024491,0.012926,-0.026501,-0.043919,0.025264,-0.053251,0.043269,0.002465,...,-0.031739,0.014192,0.008710,-0.024261,-0.004780,0.057256,0.059702,0.036648,-0.022040,0.034427
LV2,-0.045630,-0.008570,0.040969,-0.016198,-0.032321,-0.017629,0.009238,-0.011634,-0.005854,0.033497,...,0.019678,0.040569,-0.009344,-0.012921,-0.055292,0.003193,0.013844,0.008327,0.004302,0.035154
LV3,0.025341,-0.059962,-0.027354,-0.033864,0.009571,0.023318,-0.003872,0.001095,0.028179,0.003639,...,0.041022,-0.036422,0.006088,-0.041302,0.017492,-0.039591,-0.000801,0.009127,0.045741,0.004434
LV4,-0.039736,0.013238,-0.002819,0.029253,-0.029388,-0.003176,0.037212,-0.022370,0.003168,0.039998,...,-0.029197,0.078351,0.044476,-0.027300,0.005332,0.028581,0.004288,0.015797,-0.074106,-0.020936
LV5,-0.012250,-0.002846,-0.025161,0.013880,0.009457,0.024885,0.001274,0.064774,0.039294,-0.000256,...,0.032792,-0.035128,-0.011787,-0.006511,-0.035433,-0.008735,-0.011139,-0.036651,0.003044,0.006505


## eMERGE (S-MultiXcan)

In [18]:
# FIXME: in the future, there will be a specific entry in config for the eMERGE directory that should be replaced here
emerge_smultixcan_zscores_filepath = Path(
    conf.DATA_DIR,
    "emerge",
    "gene_assoc",
    "emerge-smultixcan-mashr-zscores.pkl",
).resolve()

display(emerge_smultixcan_zscores_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/data/emerge/gene_assoc/emerge-smultixcan-mashr-zscores.pkl')

In [19]:
_tmp = pd.read_pickle(emerge_smultixcan_zscores_filepath)

In [20]:
_tmp.shape

(22198, 309)

In [21]:
_tmp.head()

,292.3,079,741,418.1,280,747,591,426.21,244,070.3,...,427,585.4,345,366.2,420.2,195,250.2,250.1,371,562
gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,1.043436,1.134028,1.595979,0.399451,0.725100,1.655468,1.803598,0.125250,1.053218,0.922592,...,0.364789,0.453873,0.759558,1.173671,0.298944,1.029646,0.826238,0.685711,2.069701,1.544766
ENSG00000000457,1.236752,0.119837,0.224120,0.200610,1.304190,0.541478,0.646474,0.493759,0.189680,1.563273,...,1.519500,0.201290,1.020996,0.592006,0.632661,0.995453,0.069362,0.460979,0.217046,1.350475
ENSG00000000460,0.497108,0.005690,0.317710,0.184918,1.136574,1.708245,0.288962,1.070026,0.403510,1.106321,...,0.678253,1.166204,1.138126,0.608565,0.449551,0.096689,0.887467,0.143815,0.236647,1.048859
ENSG00000000938,0.374442,1.099899,1.975795,0.107667,0.903520,0.673819,0.855749,0.499088,0.375406,0.815757,...,1.585687,0.584143,0.751384,2.758230,0.572712,0.141023,1.437022,0.288943,0.665727,1.225229
ENSG00000000971,1.108762,1.133163,0.227188,0.222948,0.072709,0.256124,0.540911,0.606947,0.229133,1.557433,...,2.349373,1.122458,0.155346,0.990123,0.326914,1.100210,1.099537,2.511358,0.950262,1.467305


## GLS results on PhenomeXcan

Read results obtained with `30-gls_on_phenotypes.ipynb` (PhenomeXcan)

In [22]:
gls_phenomexcan = pd.read_pickle(PHENOMEXCAN_INPUT_FILE)

In [23]:
gls_phenomexcan.shape

(100, 9)

In [24]:
gls_phenomexcan.head()

,part_k,cluster_id,phenotype,lv,lv_with_pathway,coef,pvalue,pvalue_twosided,summary
0,29,12,30120_raw-Lymphocyte_count,LV644,False,0.000996,0.468635,0.937271,GLS Regression Res...
1,29,12,30120_raw-Lymphocyte_count,LV335,True,0.003096,0.405843,0.811686,GLS Regression Res...
2,29,12,30120_raw-Lymphocyte_count,LV34,True,0.045025,0.000219,0.000439,GLS Regression Res...
3,29,12,30120_raw-Lymphocyte_count,LV444,False,0.001527,0.452213,0.904427,GLS Regression Res...
4,29,12,30120_raw-Lymphocyte_count,LV705,True,0.031607,0.006947,0.013893,GLS Regression Res...


## MultiPLIER summary

In [25]:
multiplier_model_summary = pd.read_pickle(conf.MULTIPLIER["MODEL_SUMMARY_FILE"])

In [26]:
multiplier_model_summary.shape

(2157, 5)

In [27]:
multiplier_model_summary.head()

,pathway,LV index,AUC,p-value,FDR
1,KEGG_LYSINE_DEGRADATION,1,0.388059,0.866078,0.956005
2,REACTOME_MRNA_SPLICING,1,0.733057,0.000048,0.000582
3,MIPS_NOP56P_ASSOCIATED_PRE_RRNA_COMPLEX,1,0.680555,0.001628,0.011366
4,KEGG_DNA_REPLICATION,1,0.549473,0.312155,0.539951
5,PID_MYC_ACTIVPATHWAY,1,0.639303,0.021702,0.083739


In [28]:
well_aligned_lvs = multiplier_model_summary[
    (multiplier_model_summary["FDR"] < 0.05) | (multiplier_model_summary["AUC"] >= 0.75)
]

display(well_aligned_lvs.shape)
display(well_aligned_lvs.head())

(469, 5)

,pathway,LV index,AUC,p-value,FDR
2,REACTOME_MRNA_SPLICING,1,0.733057,4.772691e-05,5.816211e-04
3,MIPS_NOP56P_ASSOCIATED_PRE_RRNA_COMPLEX,1,0.680555,1.628217e-03,1.136590e-02
8,REACTOME_MITOTIC_G1_G1_S_PHASES,1,0.686170,2.517619e-04,2.392292e-03
9,IRIS_Monocyte-Day0,2,0.890036,4.315812e-25,1.329887e-22
10,DMAP_MONO2,2,0.904676,1.313970e-16,1.574574e-14


In [29]:
well_aligned_lv_codes = set([f"LV{lvi}" for lvi in well_aligned_lvs["LV index"]])

In [30]:
len(well_aligned_lv_codes)

200

In [31]:
list(well_aligned_lv_codes)[:5]

['LV932', 'LV356', 'LV954', 'LV904', 'LV767']

# Select LVs

In [32]:
gls_phenomexcan.sort_values("pvalue").head(10)

,part_k,cluster_id,phenotype,lv,lv_with_pathway,coef,pvalue,pvalue_twosided,summary
36,29,12,6152_100-Blood_clot_DVT_bronchitis_emphysema_a...,LV948,False,0.086323,3.299886e-12,6.599773e-12,GLS Regression Res...
34,29,12,6152_100-Blood_clot_DVT_bronchitis_emphysema_a...,LV96,True,0.085516,5.506865e-12,1.101373e-11,GLS Regression Res...
80,29,12,asthma,LV207,False,0.084608,1.191771e-11,2.383542e-11,GLS Regression Res...
49,29,12,6152_8-Blood_clot_DVT_bronchitis_emphysema_ast...,LV207,False,0.083574,2.052668e-11,4.105337e-11,GLS Regression Res...
73,29,12,6152_9-Blood_clot_DVT_bronchitis_emphysema_ast...,LV948,False,0.082493,2.536922e-11,5.073843e-11,GLS Regression Res...
94,29,12,asthma,LV948,False,0.076148,1.011494e-09,2.022989e-09,GLS Regression Res...
92,29,12,asthma,LV96,True,0.074659,2.171634e-09,4.343268e-09,GLS Regression Res...
30,29,12,6152_100-Blood_clot_DVT_bronchitis_emphysema_a...,LV942,True,0.072792,3.880992e-09,7.761983e-09,GLS Regression Res...
77,29,12,6152_9-Blood_clot_DVT_bronchitis_emphysema_ast...,LV96,True,0.072162,4.840323e-09,9.680647e-09,GLS Regression Res...
53,29,12,6152_8-Blood_clot_DVT_bronchitis_emphysema_ast...,LV948,False,0.071636,8.440642e-09,1.688128e-08,GLS Regression Res...


In [33]:
gls_phenomexcan_lvs = (
    gls_phenomexcan[gls_phenomexcan["pvalue"] < 0.05]
    .sort_values("pvalue")["lv"]
    .drop_duplicates(keep="first")
    .to_numpy()
)
display(gls_phenomexcan_lvs)

array(['LV948', 'LV96', 'LV207', 'LV942', 'LV705', 'LV34', 'LV525',
       'LV644', 'LV853', 'LV564', 'LV985', 'LV363', 'LV444', 'LV774',
       'LV390', 'LV679', 'LV335', 'LV586', 'LV941', 'LV882'], dtype=object)

In [34]:
len(gls_phenomexcan_lvs)

20

In [35]:
assert len(gls_phenomexcan_lvs) == len(set(gls_phenomexcan_lvs)), "Repeated LVs in list"

# GLSPhenoplier

## Get list of phenotypes/lvs pairs

In [36]:
traits = emerge_traits_df["phecode"].unique()
display(traits.shape)

(309,)

In [37]:
phenotypes_lvs_pairs = []

# for each LV that was run on PhenomeXcan, I take the top `N_TOP_TRAITS_FROM_LV` traits
# in eMERGE
# ~~~+ global mapped phenotypes (variable `gls_emerge_phecodes`)~~~
for lv in gls_phenomexcan_lvs:

    for phenotype_code in traits:  # + gls_emerge_phecodes):
        phenotypes_lvs_pairs.append(
            {
                "phenotype": phenotype_code,
                "lv": lv,
            }
        )

phenotypes_lvs_pairs = pd.DataFrame(phenotypes_lvs_pairs).drop_duplicates()

In [38]:
phenotypes_lvs_pairs = phenotypes_lvs_pairs.sort_values("phenotype").reset_index(
    drop=True
)

In [39]:
phenotypes_lvs_pairs.shape

(6180, 2)

In [40]:
phenotypes_lvs_pairs.head()

,phenotype,lv
0,008,LV948
1,008,LV96
2,008,LV335
3,008,LV679
4,008,LV207


## Run

In [41]:
results = []

pbar = tqdm(total=phenotypes_lvs_pairs.shape[0], ncols=100)

for idx, row in phenotypes_lvs_pairs.iterrows():
    phenotype_code = row["phenotype"]
    lv_code = row["lv"]

    pbar.set_description(f"{phenotype_code} - {lv_code}")

    gls_model = GLSPhenoplier(
        smultixcan_result_set_filepath=emerge_smultixcan_zscores_filepath
    )
    gls_model.fit_named(lv_code, phenotype_code)
    res = gls_model.results

    results.append(
        {
            "phenotype": phenotype_code,
            "lv": lv_code,
            "lv_with_pathway": lv_code in well_aligned_lv_codes,
            "coef": res.params.loc["lv"],
            "pvalue": res.pvalues_onesided.loc["lv"],
            "pvalue_twosided": res.pvalues.loc["lv"],
            "summary": gls_model.results_summary,
        }
    )

    # save results every 10 models trained
    if (idx % 10) == 0:
        pd.DataFrame(results).to_pickle(OUTPUT_FILENAME)

    pbar.update(1)

pbar.close()

796 - LV882: 100%|███████████████████████████████████████████| 6180/6180 [42:42:30<00:00, 24.88s/it]


In [42]:
results = pd.DataFrame(results)

In [43]:
results.shape

(6180, 7)

In [44]:
results.head()

,phenotype,lv,lv_with_pathway,coef,pvalue,pvalue_twosided,summary
0,008,LV948,False,-0.010377,0.791969,0.416062,GLS Regression Res...
1,008,LV96,True,-0.011360,0.813072,0.373857,GLS Regression Res...
2,008,LV335,True,0.004037,0.375220,0.750440,GLS Regression Res...
3,008,LV679,False,-0.016359,0.906661,0.186679,GLS Regression Res...
4,008,LV207,False,0.010934,0.195141,0.390282,GLS Regression Res...


In [45]:
results.sort_values("pvalue").head(10)

,phenotype,lv,lv_with_pathway,coef,pvalue,pvalue_twosided,summary
4225,509,LV207,False,0.059579,0.000001,0.000003,GLS Regression Res...
3371,429,LV444,False,0.051574,0.000015,0.000029,GLS Regression Res...
5212,591,LV564,False,0.051078,0.000022,0.000043,GLS Regression Res...
4681,564,LV882,False,0.046387,0.000051,0.000102,GLS Regression Res...
4072,483,LV882,False,0.046222,0.000059,0.000117,GLS Regression Res...
3716,443.9,LV644,False,0.047482,0.000066,0.000133,GLS Regression Res...
4031,480,LV882,False,0.045475,0.000069,0.000138,GLS Regression Res...
3319,428.2,LV390,False,0.047641,0.000077,0.000155,GLS Regression Res...
589,244,LV948,False,0.047649,0.000113,0.000226,GLS Regression Res...
3076,427.11,LV948,False,0.046780,0.000113,0.000227,GLS Regression Res...


## Save

In [46]:
results.to_pickle(OUTPUT_FILENAME)